In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-o1nly "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/dataset1/preprocessed.csv


In [3]:
import openpyxl
import torch
import pandas as pd
from sklearn.model_selection import train_test_split, KFold
from sklearn.metrics import accuracy_score, f1_score, matthews_corrcoef
from transformers import AutoTokenizer, AutoModelForSequenceClassification, AdamW
from torch.utils.data import DataLoader, TensorDataset
from tqdm import tqdm

In [4]:
workbook = openpyxl.Workbook()
worksheet = workbook.active
worksheet.append(["Fold", "Model", "Test Accuracy Before", "Test F1 Before", "Test MCC Before",
                  "Test Accuracy After", "Test F1 After", "Test MCC After"])

In [5]:
dataset=pd.read_csv("/kaggle/input/dataset1/preprocessed.csv")

In [6]:
dataset.head()

,tweet_id,tweet,sarcasm,processed_tweets
0,866871160725794816,Triple Talaq par Burbak Kuchh nahi bolega,0,Triple Talaq par Burbak Kuchh nahi bolega
1,880356789358743553,Batao ye uss site pr se akki sir ke verdict ni...,1,Batao ye uss site pr se akki sir ke verdict ni...
2,877751493889105920,Hindu baheno par julam bardas nahi hoga @Tripl...,0,Hindu baheno par julam bardas nahi hoga Hindu ...
3,901806457871466496,Naa bhai.. aisa nhi hai.. mere handle karne se...,0,Naa bhai .. aisa nhi hai .. mere handle karne ...
4,866264330748219392,#RememberingRajiv aaj agar musalman auraten tr...,0,Remembering Rajiv aaj agar musalman auraten tr...


In [7]:
dataset.drop(['tweet'],axis=1,inplace=True)

In [8]:
dataset['processed_tweets'][0]

'Triple Talaq par Burbak Kuchh nahi bolega'

In [9]:
template = "Categorize the following Hinglish text into one of the predefined groups: sarcastic and non-sarcastic. The text is \"////\"."

# Format the tweets into the template
dataset["processed_tweets"] = [template.replace("////", tweet) for tweet in dataset["processed_tweets"]]

In [10]:
test_models=["bert-base-multilingual-cased","distilbert-base-multilingual-cased","xlm-roberta-base","timpal0l/mdeberta-v3-base-squad2","microsoft/Multilingual-MiniLM-L12-H384"]

In [11]:

# Define evaluation function
def evaluate_model(data_loader, model, device):
    model.eval()
    predictions = []
    true_labels = []
    with torch.no_grad():
        for inputs, attention_mask, labels in data_loader:
            inputs, attention_mask, labels = inputs.to(device), attention_mask.to(device), labels.to(device)
            outputs = model(inputs, attention_mask=attention_mask)
            _, predicted = outputs.logits.max(1)
            predictions.extend(predicted.tolist())
            true_labels.extend(labels.tolist())
    accuracy = accuracy_score(true_labels, predictions)
    f1 = f1_score(true_labels, predictions)
    mcc = matthews_corrcoef(true_labels, predictions)
    return accuracy, f1, mcc

# Define training loop
def train_model(model, train_loader, optimizer, scheduler, device, num_epochs):
    for epoch in range(num_epochs):
        model.train()
        running_loss = 0.0

        for batch in tqdm(train_loader, desc=f"Epoch {epoch + 1}/{num_epochs}"):
            input_ids, attention_mask, labels = batch 
            input_ids, attention_mask, labels = input_ids.to(device), attention_mask.to(device), labels.to(device)

            optimizer.zero_grad()
            outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
            loss = outputs.loss
            loss.backward()
            optimizer.step()

            running_loss += loss.item()

        scheduler.step()

        print(f"Epoch {epoch + 1}/{num_epochs}, Loss: {running_loss / len(train_loader):.4f}")

        # Save the changes to the Excel file after each epoch
        workbook.save('finetuned_model_accu_scores2.xlsx')


# Tokenize input sequences
max_length = 128

def tokenize_data(data, tokenizer, max_length):
    tokenized_data = tokenizer(data['processed_tweets'].tolist(), padding=True, truncation=True, max_length=max_length, return_tensors='pt')
    labels = torch.tensor(data['sarcasm'].tolist())
    return tokenized_data, labels

# Split the dataset into 4 folds
num_folds = 4
kf = KFold(n_splits=num_folds)

# Initialize lists to store evaluation metrics
test_accuracy_before_list = []
test_accuracy_after_list = []
test_f1_before_list = []
test_f1_after_list = []
test_mcc_before_list = []
test_mcc_after_list = []

# Iterate over models
for i in range(0,len(test_models)): # 47th modela
    try:
        fold = 0
        for train_index, test_index in kf.split(dataset):
            fold += 1
            train_data_fold, test_data_fold = dataset.iloc[train_index], dataset.iloc[test_index]
            # Load tokenizer and model
            model_name = test_models[i]
            tokenizer = AutoTokenizer.from_pretrained(model_name)
            model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=2)

            # Move model to appropriate device
            device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
            model.to(device)

            # Define optimizer and learning rate scheduler
            optimizer = AdamW(model.parameters(), lr=2e-5)
            scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=1, gamma=0.9)

            # Tokenize and create DataLoader for training and testing data
            tokenized_train_data, train_labels = tokenize_data(train_data_fold, tokenizer, max_length)
            tokenized_test_data, test_labels = tokenize_data(test_data_fold, tokenizer, max_length)

            train_dataset = TensorDataset(tokenized_train_data['input_ids'], tokenized_train_data['attention_mask'],
                                          train_labels)
            test_dataset = TensorDataset(tokenized_test_data['input_ids'], tokenized_test_data['attention_mask'],
                                         test_labels)

            train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
            test_loader = DataLoader(test_dataset, batch_size=32)

            # Evaluate the model before training
            test_accuracy_before, test_f1_before, test_mcc_before = evaluate_model(test_loader, model, device)
            print(f"Fold {fold}, Model {model_name}:",i)
            print(f"Test Accuracy (Before fine-tuning): {test_accuracy_before:.2f}%")
            print(f"Test F1 Score (Before fine-tuning): {test_f1_before:.2f}")
            print(f"Test Matthews Correlation Coefficient (Before fine-tuning): {test_mcc_before:.2f}")

            # Train the model
            num_epochs = 10
            train_model(model, train_loader, optimizer, scheduler, device, num_epochs)

            # Evaluate the model after training
            test_accuracy_after, test_f1_after, test_mcc_after = evaluate_model(test_loader, model, device)
            print(f"Test Accuracy (After fine-tuning): {test_accuracy_after:.2f}%")
            print(f"Test F1 Score (After fine-tuning): {test_f1_after:.2f}")
            print(f"Test Matthews Correlation Coefficient (After fine-tuning): {test_mcc_after:.2f}")

            # Store evaluation metrics
            test_accuracy_before_list.append(test_accuracy_before)
            test_accuracy_after_list.append(test_accuracy_after)
            test_f1_before_list.append(test_f1_before)
            test_f1_after_list.append(test_f1_after)
            test_mcc_before_list.append(test_mcc_before)
            test_mcc_after_list.append(test_mcc_after)

            # Append the evaluation metrics to the Excel worksheet
            worksheet.append([fold, model_name, test_accuracy_before, test_f1_before, test_mcc_before,
                              test_accuracy_after, test_f1_after, test_mcc_after])

        print('Training and evaluation of model ', model_name, ' complete!')
    except Exception as e:
        print(f"Error in iteration {i}: {str(e)}")

# Calculate changes in metrics
accuracy_change = [(after - before) for before, after in zip(test_accuracy_before_list, test_accuracy_after_list)]
f1_change = [(after - before) for before, after in zip(test_f1_before_list, test_f1_after_list)]
mcc_change = [(after - before) for before, after in zip(test_mcc_before_list, test_mcc_after_list)]

# Calculate average scores after each fold
avg_accuracy_after_fold = sum(test_accuracy_after_list) / len(test_accuracy_after_list)
avg_f1_after_fold = sum(test_f1_after_list) / len(test_f1_after_list)
avg_mcc_after_fold = sum(test_mcc_after_list) / len(test_mcc_after_list)

# Print or store the results as per your requirement
print("Average Accuracy After Each Fold:", avg_accuracy_after_fold)
print("Average F1 Score After Each Fold:", avg_f1_after_fold)
print("Average Matthews Correlation Coefficient After Each Fold:", avg_mcc_after_fold)

# Save the evaluation metrics to an Excel file
workbook.save('evaluation_metrics2.xlsx')

tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/625 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/996k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.96M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/714M [00:00<?, ?B/s]

...

Epoch 8/10, Loss: 0.0450


Epoch 9/10: 100%|██████████| 124/124 [00:14<00:00,  8.73it/s]


Epoch 9/10, Loss: 0.0386


Epoch 10/10: 100%|██████████| 124/124 [00:14<00:00,  8.71it/s]


Epoch 10/10, Loss: 0.0356
Test Accuracy (After fine-tuning): 0.83%
Test F1 Score (After fine-tuning): 0.47
Test Matthews Correlation Coefficient (After fine-tuning): 0.50
Training and evaluation of model  microsoft/Multilingual-MiniLM-L12-H384  complete!
Average Accuracy After Each Fold: 0.9324755493842067
Average F1 Score After Each Fold: 0.7291933387003103
Average Matthews Correlation Coefficient After Each Fold: 0.7285809654319604
